# Training Model for ASL Words

## Collecting Data And Splitting

To get all the data, I will loop through all the videos for all the words, and get the numpy arrays of each frame I saved during the pre-processing step

In [1]:
import numpy as np
import os

### Collecting The Data

In [2]:
num_words = 3
num_videos = 100
start_frames = 5
end_frames = 75
videos_base_dir = 'video_data'

Above, I defined to start getting the frames starting from the 5th frame. This is because I found out that the cv2 module keeps recording videos even when I put a waitKey(2000) to wait 2 seconds to get into position for each video. I was not able to find any solutions or reasons why this is happening. So, I will instead just disregard the first 5 frames of each video.

In [4]:
def collect_data(words_to_collect=3, videos_per_word=100, start_frame=0, end_frame=75):
    data = []
    labels = []
    for words in range(words_to_collect):
        for videos in range(videos_per_word):
            video_array_list = []
            for frames in range(start_frame, end_frame):
                numpy_array_file_path = os.path.join(videos_base_dir,f'{words}',f'{videos}', 'landmarks', f'{words}_{videos}_{frames}.npy')
                temp_array = np.load(numpy_array_file_path)
                #print(f'Loading {numpy_array_file_path}')
                video_array_list.append(temp_array)
            data.append(video_array_list)
            labels.append(words)
    return data,labels
        

To check if this worked, I will print the shape of the data and labels and also look at some of the data arrays

In [5]:
data, labels = collect_data(words_to_collect=num_words, videos_per_word=num_videos, start_frame=start_frames, end_frame=end_frames)

In [6]:
data_np = np.array(data)
print(f'Shape of data: {data_np.shape}')

labels_np = np.array(labels)
print(f'Shape of labels: {labels_np.shape}')

Shape of data: (300, 70, 1662)
Shape of labels: (300,)


The shape seems right. I will do check the data to make sure everything was converted correctly

In [7]:
for word in range(num_words):
    for video in range(num_videos):
        for frame in range(start_frames, end_frames):
            if np.sum(data_np[video + (word*100)][frame-5] != np.load(os.path.join(videos_base_dir,f'{word}',f'{video}', 'landmarks', f'{word}_{video}_{frame}.npy'))):
                print(f'Error at {word}, {video}, {frame}')

In [8]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


So this means all of the data was loaded successfully

### Randomly Splitting The Data

The next step is randomizing this data and then splitting it into 3 sections. Training data, Validation data, and Testing data.

In [9]:
rng = np.random.RandomState(113)
data_indices = np.arange(len(data))
#print(data_indices)
rng.shuffle(data_indices)

In [10]:
print(data_indices)

[151  79 268 198 177   6  92 269  34  12 233 144 175 259  32  33  40 231
 255  96 132  21  71 181  19  22 152 146  35 115 142 154 156 157 148 117
 272 133  62 141   3  56 143  63 221 147 182 210 145 260  46 298 294 257
 174  43 124  29 126 171  16  25  80  30  41  91 106  26 114 108  93  20
 158 120   0 119 212 219 134 299  44 168  82 229 234   5 105 286 129 247
 285 297 125  17  68 178  81 137 155 275   9 203  86  39  28 209 238 187
 251 192 250  94 195   8  78 283  24 236 256  42 131 226 169 123 293 190
 227 196 204 109  14 262  59 202  50  57 225  66 176 241 217   2 113  18
 200 159 166 264 103  13  51  84  58 107 237 140   4 121  48 104 111 205
  60 271 163 161  45 265  27 191 127 180  77  65 248 201  72  37 295  61
 110 130  98  53  99 223 214  75 232  10  54 164 128 179  90 292  52 188
 270 136 135 197 289  38 224 101  89 138 242 208 207 243 211 170 273 100
 239 263 206 118  49  15  76 216 153 194 291 112 235 172 160  87 290 193
  88  83 122 184 278 246  31 139 186  70 296 287 10

Now I have a randomized indices array. I can split this array into 3, and then get the data at these indices for each section. The split will be 70% training, 15% validation, and 15% testing

In [11]:
train_indices = data_indices[0:210]
val_indices = data_indices[210:255]
test_indices = data_indices[255:300]

In [12]:
x_train = data_np[train_indices]
y_train = labels_np[train_indices]

x_val = data_np[val_indices]
y_val = labels_np[val_indices]

x_test = data_np[test_indices]
y_test = labels_np[test_indices]

I will do some quick manual checks by looking at the randomized indices that were generated

In [13]:
print(x_train[5][5] == np.load('video_data/0/6/landmarks/0_6_10.npy'))
print(y_train[5])

[ True  True  True ...  True  True  True]
0


In [14]:
print(x_val[41][15] == np.load('video_data/2/82/landmarks/2_82_20.npy'))
print(y_val[41])


[ True  True  True ...  True  True  True]
2


In [15]:
print(x_test[44][50] == np.load('video_data/1/65/landmarks/1_65_55.npy'))
print(y_test[44])

[ True  True  True ...  True  True  True]
1


Ok it seems like the random shuffling has worked

## Creating And Training The Model

Now that I have training, testing, and validation data, I will create a model, compile it, and then train it using this data. Then I will run some predictions using the testing dataset.

### Creating A Model

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [38]:
model = Sequential([
    LSTM(32, return_sequences=True, activation='relu', input_shape=(70,1662)),
    LSTM(32, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

In [39]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                   │ (None, 70, 32)         │       216,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,435 (884.51 KB)

 Trainable params: 226,435 (884.51 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)

In [33]:
print(type(y_train_cat))
print(y_train_cat.shape)

<class 'numpy.ndarray'>
(210, 3)


In [27]:
type(x_train)

numpy.ndarray

In [31]:
type(model)

keras.src.models.sequential.Sequential

In [41]:
model.fit(x_train, y_train_cat, epochs=50, batch_size=32)

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.3764 - loss: 1.2027
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3176 - loss: 1.1829
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3752 - loss: 1.1330
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3251 - loss: 1.1404
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3705 - loss: 1.1019
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3627 - loss: 1.0931
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3105 - loss: 1.1170
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3259 - loss: 1.1031
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.3902 - loss: 1.0759
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4661 - loss: 1.0843
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5075 - loss: 1.0807
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4075 - loss: 1.0671
E

In [42]:
model.evaluate(x_test, to_categorical(y_test))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9095 - loss: 0.2993 


[0.29621949791908264, 0.9111111164093018]

So, the model currently has 94% accuracy on the training data, and 91% accuracy on the testing data.

This means the model is working. Next step is to save this model so it can be used in other applications, such as in real-time using a web-cam to live-translate some sign language words

In [43]:
model.save(filepath='asl_words_model_91acc.keras')

Another example of testing one video

In [52]:
np.argmax(model.predict(x_test[0].reshape((1,70,1662))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


1

This means the model predicted that this video is for the second word: __drink__

To check, I will get the index for the first video in the testing dataset

In [53]:
print(test_indices[0])

173


This means that the video in the first index of the testing dataset is the 73rd video of the second word: __drink__. Since each word is 100 videos with 0-99 being for the word __book__, 100-199 for the word __drink__, and 200-299 for the word __computer__